# Projeto 1 - Ciência dos Dados

Nome: Maria Eduarda Torres

Nome: Giancarlo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import numpy
import itertools
import collections
import functools
import operator
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gvgia\Documents\GitHub\NAIVE-BAYES


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'iphone_12.xlsx'

In [4]:
train = pd.read_excel(filename)
train

,Treinamento,categorias
0,até junho compro meu iphone 12 pro max se deus...,1
1,lembrando do dia que eu derrubei um iphone 12 ...,0
2,alguém me da um iphone 12 ai pfv,1
3,vem projota quero ganhar um iphone 12 que eu p...,1
4,"saí mt lindo na foto da identidade mane, camer...",1
...,...,...
295,quem ta afim de me dar um iphone 12 pro max 51...,0
296,entrei em um sorteio pra ganhar um iphone 12 n...,0
297,"@jornaloglobo legal fera, agora é só esperar o...",0
298,@risk_protection comprar iphone 12 o galaxy,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,categorias
0,"iphone 12 mini é um iphone 5s, por fora né",1
1,eu sou trouxa ao ponto de ter esperança de gan...,0
2,"ótima comparação, po. esse notebook custa apen...",1
3,@__karlamaiolly de brinde vem uma carne e um i...,1
4,eu quero! clique no link para me ajudar a baix...,0
...,...,...
231,"nem iphone 12, nem whisky 12 anos, nem carro z...",1
232,iplace me mandando e-mail sobre iphone 12........,1
233,"@mii_silvaaa nss eu tb, só qria um iphone 12 😔...",1
234,"rt @jocecrz: @jornaloglobo legal fera, agora é...",1


****

___
# <kbd><code style="font-family:Times New Roman;background:black;color:white">Classificador automático de sentimento</code></kbd>


<u>*Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.*</u>

*O produto escolhido foi o "Iphone 12". Classificamos como relevante todos os tweets que traria algum feedback, tanto positivo como negativo sobre o celular para que a empressa pudesse saber no que melhorar. Os tweets que falavam sobre aposta, sorteio ou comparaco com outros produtos que não são concorrentes foram classificados como irrelevantes.*

***

___
# <kbd><code style="font-family:Times New Roman;background:black;color:white">Montando um Classificador Naive-Bayes</code></kbd>



<code style="font-family:Times New Roman">Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.</code>

****
> <code style="background:darkblue;color:white">Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc</code>

In [6]:
import re 

def cleanup(text):
    """ 
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:_?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def clean_and_replace(list_to_clean):
    lista = []
    for e in list_to_clean:
        c = cleanup(e.lower())
        x = re.sub("@[A-Za-z0-9]+","",c)
        z = re.sub("https//tco/[A-Za-z0-9]+","",x)
        y = z.replace('\n', '').replace('  ', ' ')
        lista.append(y)
    return lista

<mark>*Texto total*</mark>

`lista_T` $\Rightarrow$ lista de tweets já limpos para treinamento 

In [7]:
lista_T=[] 
lista_T = clean_and_replace(train.Treinamento)

print("O total de tweets para treinamento do classificador é:",len(lista_T))

O total de tweets para treinamento do classificador é: 300


`texto_total` $\Rightarrow$ *Reunindo todos os tweets em uma única variável de texto, separando cada palavra por um espaço*

In [8]:
texto_total=""
for e in lista_T:
    texto_total+= " " + e 
texto_total = cleanup(texto_total.lower())

<mark>*Texto relevante*</mark>

`relevante` $\Rightarrow$ *Separando os tweets de Treinamento da categoria Relevante*

In [9]:
filtra_linhas= train['categorias'] == 1
relevante= train.loc[filtra_linhas, :]

`Lista_T` $\Rightarrow$ *lista de tweets relevantes já limpos para treinamento*

In [10]:
lista_R=[]    
lista_R = clean_and_replace(relevante.Treinamento)

print('O total de tweets relevantes para treinamento do classificador é:',len(lista_R))

O total de tweets relevantes para treinamento do classificador é: 146


`texto_R` $\Rightarrow$ *Reunindo todos os tweets relevantes em uma única variável de texto, separando cada palavra por um espaço*

In [11]:
texto_R=""
for e in lista_R:
    texto_R+= " " + e 
texto_R = cleanup(texto_R.lower())

<mark>*Texto irrelevante*</mark>

`irrelevante` $\Rightarrow$ *Separando os tweets de Treinamento da categoria Irrelevante*

In [12]:
filtra_linhas_2 = train['categorias'] == 0
irrelevante= train.loc[filtra_linhas_2, :]

`Lista_I` $\Rightarrow$ *lista de tweets irrelevantes já limpos para treinamento*

In [13]:
lista_I=[]    
lista_I = clean_and_replace(irrelevante.Treinamento)

print('O total de tweets irrelevantes para treinamento do classificador é:',len(lista_I))

O total de tweets irrelevantes para treinamento do classificador é: 154


`texto_R` $\Rightarrow$ *Reunindo todos os tweets irrelevantes em uma única variável de texto, separando cada palavra por um espaço*

In [14]:
texto_I =""   
for e in lista_I:
    texto_I += " " + e 
texto_I = cleanup(texto_I.lower())

****

****
> <code style="background:darkblue;color:white">Correção de espaços entre palavras e/ou emojis</code>

`texto` $\Rightarrow$ *texto contendo todas as palavras de treinamento após a correção de espaços*

In [15]:
texto_split_emoji = emoji.get_emoji_regexp().split(texto_total)
texto_split_whitespace = [substr.split() for substr in texto_split_emoji]
texto = functools.reduce(operator.concat, texto_split_whitespace)

`texto_relevante` $\Rightarrow$ *texto contendo apenas as palavras de treinamento do subconjunto relevante após a correção de espaços*

In [16]:
texto_split_emoji_R = emoji.get_emoji_regexp().split(texto_R)
texto_split_whitespace_R = [substr.split() for substr in texto_split_emoji_R ]
texto_relevante = functools.reduce(operator.concat, texto_split_whitespace_R)

`texto_irrelevante` $\Rightarrow$ *texto contendo apenas as palavras de treinamento do subconjunto irrelevante após a correção de espaços*

In [17]:
texto_split_emoji_I = emoji.get_emoji_regexp().split(texto_I)
texto_split_whitespace_I = [substr.split() for substr in texto_split_emoji_I ]
texto_irrelevante = functools.reduce(operator.concat, texto_split_whitespace_I)

***

****
> <code style="background:darkblue;color:white">Suavização de Laplace</code>

<mark>*Cálculo das Probabilidades*</mark>

`prob_relevante` $\Rightarrow$ *Probabilidade do tweet ser relevante*

In [18]:
prob_relevante = len(lista_R)/len(lista_T)
print("A probabilidade do tweet ser relevante: P(R) =",prob_relevante)

A probabilidade do tweet ser relevante: P(R) = 0.4866666666666667


`prob_irrelevante` $\Rightarrow$ *Probabilidade do tweet ser irrelevante*

In [19]:
prob_irrelevante = len(lista_I)/len(lista_T)
print("A probabilidade do tweet ser irrelevante: P(I) =",prob_irrelevante)

A probabilidade do tweet ser irrelevante: P(I) = 0.5133333333333333


`total_texto_relevante` $\Rightarrow$ *Total de palavras no subconjunto relevante*

In [20]:
total_texto_relevante = len(texto_relevante)

`total_texto_irrelevante` $\Rightarrow$ *Total de palavras no subconjunto irrelevante*

In [21]:
total_texto_irrelevante = len(texto_irrelevante)

`vocabulario` $\Rightarrow$ *Permite obter o total de palavras diferentes no vocabulário de treinamento*

In [22]:
vocabulario = pd.Series(texto).drop_duplicates()

`total_por_palavra_R` $\Rightarrow$ *Contagem das palavras relevantes*

In [23]:
total_por_palavra_R = pd.Series(texto_relevante).value_counts()

`total_por_palavra_I` $\Rightarrow$ *Contagem das palavras irrelevantes*

In [24]:
total_por_palavra_I = pd.Series(texto_irrelevante).value_counts()

<mark>*Cálculo das probabilidades das palavras do vocabulário*</mark>

In [25]:
prob_palavras_R=[]
prob_palavras_I=[]

## Suavização de Laplace: Caso não tenha a palavra no vocabulario
# Probabilidade no conjunto relevante
laplace_R = 1/(total_texto_relevante + len(vocabulario))

#Probabilidade no conjunto irrelevante
laplace_I = 1/(total_texto_irrelevante + len(vocabulario))

#Percorre todo o vocabulario calculando as probabilidades tanto relevante como irrelevante
for palavra in vocabulario:
    if palavra in total_por_palavra_R:
        prob_r = (total_por_palavra_R[palavra] + 1)/(total_texto_relevante + len(vocabulario))
        prob_palavras_R.append(prob_r)
    else:
        prob_palavras_R.append(laplace_R)

    if palavra in total_por_palavra_I:
        prob_i = (total_por_palavra_I[palavra] + 1)/(total_texto_irrelevante + len(vocabulario))
        prob_palavras_I.append(prob_i)
    else:
        prob_palavras_I.append(laplace_I)

<mark>*Organização dos dados: dataframe com as palavras e probabilidades*</mark>

In [26]:
lista_prob = list(zip(vocabulario,prob_palavras_R,prob_palavras_I))
df_prob_aux = pd.DataFrame(lista_prob, columns = ['Palavra','P(palavra|Relevante)','P(palavra|Irrelevante)'])
df_prob = df_prob_aux.set_index('Palavra')
print(df_prob)

           P(palavra|Relevante)  P(palavra|Irrelevante)
Palavra                                                
até                    0.001569                0.001513
junho                  0.000523                0.000432
compro                 0.000523                0.004107
meu                    0.004969                0.005620
iphone                 0.039487                0.031777
...                         ...                     ...
esperar                0.000262                0.000432
metade                 0.000262                0.000432
financiar              0.000262                0.000432
rlk                    0.000262                0.000432
chama                  0.000262                0.000432

[1463 rows x 2 columns]


***

# <kbd><code style="font-family:Times New Roman;background:black;color:white">Verificando a performance do Classificador</code></kbd>



<code style="font-family:Times New Roman">Agora você deve testar o seu classificador com a base de Testes.</code>

****
> <code style="background:darkblue;color:white">Limpeza de mensagens removendo os caracteres da base teste: enter, :, ", ', (, ), etc</code>

`lista_R_teste` $\Rightarrow$ *Limpeza da Lista para Teste de Tweets Relevantes*

In [27]:
filtra_linhas_teste= test['categorias'] == 1
relevante_teste= test.loc[filtra_linhas_teste, :]
lista_R_teste = []
lista_R_teste = clean_and_replace(relevante_teste.Teste)
print("O total de tweets relevantes usados no teste do classificador é :",len(lista_R_teste))

O total de tweets relevantes usados no teste do classificador é : 102


`lista_I_teste` $\Rightarrow$ *Limpeza da Lista para Teste de Tweets Irrelevantes*

In [28]:
filtra_linhas_teste_2 = test['categorias'] == 0
irrelevante_teste= test.loc[filtra_linhas_teste_2, :]
lista_I_teste=[]
lista_I_teste = clean_and_replace(irrelevante_teste.Teste)
print("O total de tweets irrelevantes usados no teste do classificador é :",len(lista_I_teste))

O total de tweets irrelevantes usados no teste do classificador é : 134


***

****
> <code style="background:darkblue;color:white">Correção de espaços entre palavras e/ou emojis da base teste</code>

`lista_R` $\Rightarrow$ *lista de tweets relevantes já limpos para teste*

In [29]:
lista_R=[] 
for e in lista_R_teste:
    e = e.lower()
    texto_split_emoji_teste = emoji.get_emoji_regexp().split(e)
    texto_split_whitespace_teste = [substr.split() for substr in texto_split_emoji_teste]
    list_teste = functools.reduce(operator.concat, texto_split_whitespace_teste)
    lista_R.append(list_teste)

`lista_I` $\Rightarrow$ *lista de tweets irrelevantes já limpos para teste*

In [30]:
lista_I=[]
for e in lista_I_teste:
    e = e.lower()
    texto_split_emoji_teste = emoji.get_emoji_regexp().split(e)
    texto_split_whitespace_teste = [substr.split() for substr in texto_split_emoji_teste]
    list_teste = functools.reduce(operator.concat, texto_split_whitespace_teste)
    lista_I.append(list_teste)

***

****
> <code style="background:darkblue;color:white">Testando cassificador</code>

<mark>*Teste dos tweets relevantes*</mark>

In [31]:
## Teste dos tweets relevantes
resultado_classificador_R = []
hipotese_melhoria = []

k = 0

#probabilidade da mensagem ser relevante/irrelevante
prob_msg_relevante = 1      
prob_msg_irrelevante = 1

#Contagem de mensagens relevantes/irrelevantes
relevantes_corretas = 0
falso_negativo = 0

while k < len(lista_R):
    for palavra in lista_R[k]:
        #Produto das probabilidades de cada palavra do tweet
        #Caso a palavra faça parte do vocabulário de Treinamento
        try:
            prob_palavra_dado_r = df_prob.loc[palavra]['P(palavra|Relevante)']
            prob_palavra_dado_i = df_prob.loc[palavra]['P(palavra|Irrelevante)']
                
        #Exceção: caso a palavra não conste no vocabulário obtido no Treinamento, deve ter sua prob = 1/(total de palavras + vocabulario)
        except:
            prob_palavra_dado_r = laplace_R
            prob_palavra_dado_i = laplace_I

        prob_msg_relevante = prob_msg_relevante*prob_palavra_dado_r
        prob_msg_irrelevante = prob_msg_irrelevante*prob_palavra_dado_i
        
    #Após as multiplicações das probabilidades das palavras, devemos multiplicar por P(Relevante) e P(Irrelevante)
    prob_msg_relevante = prob_msg_relevante*prob_relevante
    prob_msg_irrelevante = prob_msg_irrelevante*prob_irrelevante
    
    #Classificação: se P(tweet|R).P(R) > P(tweet|I).P(I), então classificamos como Relevante.
    if prob_msg_relevante > prob_msg_irrelevante:
        relevantes_corretas+=1
        resultado_classificador_R.append("Relevante")
    else:
        falso_negativo+=1
        resultado_classificador_R.append("Irrelevante")
    
    hipotese_melhoria.append(prob_msg_relevante/prob_msg_irrelevante)
    
    k+=1
    #Após classificar um tweet, as probabilidades devemos reatribuir 1 para as probabilidades
    prob_msg_relevante = 1
    prob_msg_irrelevante = 1

print("O teste apresentou ", relevantes_corretas, 
      " mensagens relevantes corretamente identificadas e ", falso_negativo, " falso(s) negativo(s)")

O teste apresentou  74  mensagens relevantes corretamente identificadas e  28  falso(s) negativo(s)


<mark>*Teste dos tweets irrelevantes*</mark>

In [32]:
## Teste dos tweets Irrelevantes
resultado_classificador_I = []

k = 0

#probabilidade da mensagem ser relevante/irrelevante
prob_msg_relevante = 1      
prob_msg_irrelevante = 1

#Contagem de mensagens relevantes/irrelevantes
irrelevantes_corretas = 0
falso_positivo = 0

while k < len(lista_I):
    for palavra in lista_I[k]:
        #Produto das probabilidades de cada palavra do tweet
        #Caso a palavra faça parte do vocabulário de Treinamento
        try:
            prob_palavra_dado_r = df_prob.loc[palavra]['P(palavra|Relevante)']
            prob_palavra_dado_i = df_prob.loc[palavra]['P(palavra|Irrelevante)']
                
        #Exceção: caso a palavra não conste no vocabulário obtido no Treinamento, deve ter sua prob = 1/(total de palavras + vocabulario)
        except:
            prob_palavra_dado_r = laplace_R
            prob_palavra_dado_i = laplace_I

        prob_msg_relevante = prob_msg_relevante*prob_palavra_dado_r
        prob_msg_irrelevante = prob_msg_irrelevante*prob_palavra_dado_i
        
    #Após as multiplicações das probabilidades das palavras, devemos multiplicar por P(Relevante) e P(Irrelevante)
    prob_msg_relevante = prob_msg_relevante*prob_relevante
    prob_msg_irrelevante = prob_msg_irrelevante*prob_irrelevante
    
    #Classificação: se P(tweet|R).P(R) > P(tweet|I).P(I), então classificamos como Relevante.
    if prob_msg_relevante > prob_msg_irrelevante:
        falso_positivo+=1
        resultado_classificador_I.append("Relevante")
    else:
        irrelevantes_corretas+=1
        resultado_classificador_I.append("Irrelevante")
    
    hipotese_melhoria.append(prob_msg_relevante/prob_msg_irrelevante)
    
    k+=1
    #Após classificar um tweet, as probabilidades devemos reatribuir 1 para as probabilidades
    prob_msg_relevante = 1
    prob_msg_irrelevante = 1

print("O teste apresentou ", irrelevantes_corretas, 
      " mensagens irrelevantes corretamente identificadas e ", falso_positivo, " falso(s) positivo(s)")

O teste apresentou  53  mensagens irrelevantes corretamente identificadas e  81  falso(s) positivo(s)


<mark>*Cálculo das porcentagens do Classificador*</mark>

In [33]:
total_teste = len(lista_R) + len(lista_I)
porc_positivos_corretos = relevantes_corretas/total_teste
porc_negativos_corretos = irrelevantes_corretas/total_teste
porc_falso_negativo = falso_negativo/total_teste
porc_falso_positivo = falso_positivo/total_teste

print('{:.1%}'.format(porc_positivos_corretos), "de Verdadeiros Positivos")
print('{:.1%}'.format(porc_negativos_corretos), "de Verdadeiros Negativos")
print('{:.1%}'.format(porc_falso_negativo), "de Falsos Negativos")
print('{:.1%}'.format(porc_falso_positivo), "de Falsos Positivos")

31.4% de Verdadeiros Positivos
22.5% de Verdadeiros Negativos
11.9% de Falsos Negativos
34.3% de Falsos Positivos


***

___
# <kbd><code style="font-family:Times New Roman;background:black;color:white">Concluindo</code></kbd>

___
# <kbd><code style="font-family:Times New Roman;background:black;color:white"> Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste</code></kbd>



<code style="font-family:Times New Roman">Caso for fazer esse item do Projeto</code>

In [40]:
relacao_colunas = {'Teste':'Treinamento'}
df_teste_treinamento = pd.concat([train, test.rename(columns=relacao_colunas)])
pd.set_option('display.max_row', 1000)
df_embaralhado=df_teste_treinamento.sample(frac=1).reset_index(drop=True)
df_embaralhado

,Treinamento,categorias
0,"quem tem seu iate, sua ilha, os sonegadores de...",1
1,🍎 🍏 esse é pra quem tem bala na agulha ...,0
2,41 mil reais = 1 iphone 12 + 1 macbook air 512...,0
3,por esse preço eu compro um iphone 12 um macbo...,0
4,quem ta afim de me dar um iphone 12 pro max 51...,0
5,tomara que sua inveja vire um iphone 12 pro ma...,1
6,@azksnialler não vai ter eu ganhando um iphone...,0
7,e o senhorzinho de uns 80 anos ostentando seu ...,1
8,@nanicv @assumps iphone 12 não é pra qlqr um né,1
9,@willkaso vem um iphone 12,0


In [35]:
df_train,df_test=train_test_split(x,y,test_size=0.2)
x_train.head()

NameError: name 'train_test_split' is not defined

___
# <kbd><code style="font-family:Times New Roman;background:black;color:white"> Aperfeiçoamento</code></kbd>
Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
# <kbd><code style="font-family:Times New Roman;background:black;color:white"> Referências</code></kbd>

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**